['IN', 'Europe-West', 'UNKNOWN', 'US', 'INDIA', 'APEM-Indonesia',

'APEM-Rest of Asia', 'LAS', 'Brazil', 'JP', 'BR', 'Europe-East',

'APEM-META', 'North America', 'APEM-JANZ', 'IT', 'UK', 'ANZ','CAP']

In [15]:
# df = pd.DataFrame(rows)
# df.columns = field_names
# df.to_csv('indian_in.csv',index = False)

In [16]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings

from fuzzywuzzy import fuzz
from difflib import SequenceMatcher
from collections import defaultdict
import emoji
warnings.filterwarnings("ignore")

In [17]:
import networkx as nx
oo = float('inf')
NaN = float("nan")

In [18]:
# for exam
def search(df,target):
    cols = ['device_id','phone','email']
    i = 0
    while i < 3:
        # search every columns
        res = df[df[cols[i]] == target]
        # if there is at least one value 
        if len(res) > 0:
            # give the value for output
            output = res
        i+=1
    return output

In [121]:
import random

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def generate_111(x):
    s = '1'
    i=0
    while i < len(x)-1:
        s+='1'
        i+=1
    return int(s)

def tag_phone(x):
    
    # remove all symbols in phone`s input
    d = { ' ': '', '+': '', '.': '', '-': '', ';': '', '_': '', '&amp': ''}
    x = replace_all(str(x),d)
    try:
        x.upper().index('TEST')
        x = NaN
            
    except ValueError:
        try:
            # it is pure numbers?
            x = str(int(x))

            # if the number too small
            if len(str(x)) < 8:
                x = NaN

            elif (str(x)[:4] == '1234') | (int(str(x)[:3]) %  int('111') == 0):
                x = NaN
            elif len(str(x)) > 13:
                x = NaN

        # if it is not pure number
        except ValueError:

            if len(str(x)) < 8:
                x = NaN

            # if len > 19, ususally there are two phone numbers
            else: 
                x = ''.join([i for i in x if i.isdigit()])
                if len(x) > 13:
                    if (str(x)[:4] == '1234') | (int(str(x)[:3]) %  int('111') == 0):
                        x = NaN
                    else:
                        x = x[:13]
                elif len(x) < 8:
                    x = NaN
                else:
                    x = x
        try:
            float(x)
        except ValueError:
            x = NaN
    return x

def tag_email(x): 
    
    # remove all symbols in email`s input
    d = { ' ': '', '+': '', '-': '', ';': '', '_': '', '&amp': '','www.':''}
    x = replace_all(str(x),d)
    
    try: 
        # if there is @
        split = str(x).index('@')
        char = x[:split]
#         char = ''.join([i for i in x if i.isdigit() == False])
        # there is only one char in front of @
        if len(char) == 1:
            x = NaN
        else:
            x = x
    # if there is no @
    except ValueError:
        # use nan to mark it as invalid 
        x = NaN

    return x

def len15(num):
    if len(num[-15:])==15:
        x = num[-15:]
    else:
        x = NaN
    return x

def tag_device(x):
    
    # remove all symbols in device`s input
    d = { ' ': '', '+': '', '.': '', '-': '', ';': '', '_': '', '&amp': ''}
    x = replace_all(str(x),d)
    try:
        str(x).upper().index('TEST')
        x = NaN
    except ValueError:
        # if len < 4, ususally it is not a device number
        if len(str(x)) < 4 :
            # use nan to mark it as invalid 
            x = NaN
        else:
            try:
                # it is pure number
                num = str(int(x))
                # if yes, slice top 15
                x = len15(num)
            # if not pure number because of type
            except TypeError:
                # use a random number to mark it invalid 
                x = NaN
            except OverflowError:
                x = NaN
            # if not pure number because of value
            except ValueError:
                num = ''.join([i for i in x if i.isdigit()])
                try:
                    num.upper().index('IMEI')
                    x = len15(num)
                except ValueError:
                    x = tag_dvc(num)
                try:
                    num.upper().index('SLOT')
                    x = len15(num)
                except ValueError:
                    x = tag_dvc(num)
    return x

def tag_dvc(num):
    if (len(str(num)) >= 16) and ((str(num)[:2] == '35') | (str(num)[:2] == '86')):
        x = len15(num)
    else:
        x = NaN
    return x

def tag_name(x):
    
    # remove all symbols in device`s input
    d = { ' ': '','[emoji]':'', '+': '', '.': '', '-': '', ';': '', '_': '', '&amp': ''}
    x = replace_all(str(x),d)
    
    # if nan
    if (str(x).lower() == 'nan') | (str(x).lower() == 'none'):
        x = NaN
    else:
        x = x
    return x

# on real dataset

In [122]:
def prepare(df):
    df = df.drop_duplicates()
    df['phone'] = df['phone'].apply(lambda x: tag_phone(x))
    df['email'] = df['email'].apply(lambda x: tag_email(x))
    df['device_id'] = df['device_id'].apply(lambda x: tag_device(x))
    print(df.info())
#     df['customer_name'] = df['customer_name'].apply(lambda x: tag_name(x))
    return df

In [123]:
%%time
dfs = pd.read_csv(r'D:\20220119161412192.csv')
# dfs = pd.read_csv('indian_in.csv')
dfs.info()
# dfs = dfs[['phone','email','device_id']][::4]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453041 entries, 0 to 453040
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   email      59524 non-null   object
 1   phone      251935 non-null  object
 2   device_id  240110 non-null  object
dtypes: object(3)
memory usage: 10.4+ MB
Wall time: 451 ms


In [124]:
dfs = prepare(dfs)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312051 entries, 0 to 453040
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   email      40728 non-null   object
 1   phone      109373 non-null  object
 2   device_id  227980 non-null  object
dtypes: object(3)
memory usage: 9.5+ MB
None


In [125]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312051 entries, 0 to 453040
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   email      40728 non-null   object
 1   phone      109373 non-null  object
 2   device_id  227980 non-null  object
dtypes: object(3)
memory usage: 9.5+ MB


In [92]:
# dfs.info()

In [126]:
def find_vendor(col,num): 
    ls = []
    for k,v in col.value_counts().items():
        if v >= num:
#             print(k,v)
            ls.append(k)
    return ls

In [127]:
# transfer vendor information to random number 
# keep other values on the same row
def exclude_vendor(data):    
    
    df = data[['phone','email','device_id']].drop_duplicates()
    vp = find_vendor(df['phone'], 3) 
    ve = find_vendor(df['email'], 3)
    vd = find_vendor(df['device_id'], 4)

    df_vp = df[df['phone'].isin(vp)].sort_values(by='phone')
    df_ve = df[df['email'].isin(ve)].sort_values(by='email')
    df_vd = df[df['device_id'].isin(vd)].sort_values(by='device_id')
    df_sub = pd.concat([df_vp,df_ve,df_vd]).drop_duplicates()
    df_vendor = df_sub
    df_rest = df[df.index.isin(df_sub.index)==False]
    
    d1 = df_sub[df_sub['phone'].isin(vp)]
    d2 = df_sub[df_sub['phone'].isin(vp)==False]
    d1['phone'] = NaN
    df_sub = pd.concat([d1,d2])
    
    d1 = df_sub[df_sub['email'].isin(ve)]
    d2 = df_sub[df_sub['email'].isin(ve)==False]
    d1['email'] = NaN
    df_sub = pd.concat([d1,d2])
    
    d1 = df_sub[df_sub['device_id'].isin(vd)]
    d2 = df_sub[df_sub['device_id'].isin(vd)==False]
    d1['device_id'] = NaN
    
    df_sub = pd.concat([d1,d2])
    df = pd.concat([df_rest,df_sub])
    
    df['null_count'] = df.isnull().sum(axis=1)
    df = df.sort_values('null_count', ascending=True).reset_index()
    df = df[df['null_count'] < 3]
    
    df_one = df[df['null_count'] == 2]
    df = df[df['null_count'] < 2]
    df_one = df_one.drop(['null_count','index'], axis=1).drop_duplicates()
    df = df.drop(['null_count','index'], axis=1).drop_duplicates()
    print(df.info())
    return df, df_one, df_vendor

In [128]:
%%time
dt, df_one, df_vendor = exclude_vendor(dfs)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50638 entries, 0 to 50643
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   phone      49495 non-null  object
 1   email      35420 non-null  object
 2   device_id  30987 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB
None
Wall time: 1.26 s


In [129]:
df_vendor[:20]

,phone,email,device_id
419818,3212223249,vinodgm@moto.com,355573092471785
419832,3212223249,NaN,355573092471785
419826,3212223249,vin@gmail.com,355573092471785
419825,3212223249,vinodgm@mot.com,355573092471785
419803,3212223249,vinod@gmail.com,355573092471785
419805,3212223249,vinodgm@motorola.com,355573092471785
419813,3212223249,vinodgm@motorla.com,355573092471785
419817,3212223249,vinodg@g.com,355573092471785
419821,3212223249,vino@gmail.com,355573092471785
419820,3212223249,vinodgm@gmail.com,355573092471785


In [130]:
# search(dt,'3212223249')

# link node

In [131]:
def add(G,u,v,i):
    if pd.isna(u) | pd.isna(v):
        pass
    else:
        G.add_edge(u,v,weight=i)
    return
        
def link_data(df):
    G = nx.Graph() 

    for i in df.index:
        add(G,df['phone'][i], df['email'][i],i)
        add(G,df['email'][i], df['device_id'][i],i)
        add(G,df['device_id'][i],df['phone'][i],i)
        
    S = [G.subgraph(c).copy() for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    # graph = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    
    return G,S

In [132]:
%%time
G,S = link_data(dt)

Wall time: 6.41 s


# identify and output

In [133]:
def be_char(char):
    try:
        # get strings before @
        split = str(char).index('@')
        char = char[:split]
    except ValueError:
        try:
            float(char)
            pass
        except ValueError:
            char = char
        except TypeError:
            pass
    return char


def user_identify(dd,i): # O(4)
    similar = []
    if len(dd) < 2:
        pass
    else:
        if pd.isna(dd['device_id'].iloc[i]):
            pass
        elif dd['device_id'].iloc[i-1] == dd['device_id'].iloc[i]:
            similar.append(7)

        if pd.isna(dd['phone'].iloc[i]):
            pass
        elif dd['phone'].iloc[i-1] == dd['phone'].iloc[i]:
            similar.append(6)
        
        if pd.isna(dd['email'].iloc[i]):
            pass
        elif fuzz.QRatio(be_char(dd['email'].iloc[i-1]),
                       be_char(dd['email'].iloc[i])) > 80:
            similar.append(5)
            
        if sum(similar) > 10:
            out = dd.iloc[i-1:i+1]
            if len(out) >= 2:
                return out
            else:
                pass
        else:
            pass
    

def by_score(data):
    try:
        len(data)
        cells = []
        for j in range(0,len(data)): #O(len(cell))
            c = user_identify(data,j) 
            try:
                len(c)
                cells.append(c)
            except TypeError:
                pass
        return cells
    except TypeError:
        pass

def classify_char(arr):
# this function is used to classify different types of values
# then output a complete chain: cell_1: phone ,cell2: email, cell_3: device_id

    list_phone = []
    list_email = []
    list_device = []
    list_index = []
    
    for char in arr:
        try:
            char = char - 0
            list_index.append(char)
        except TypeError:
            try:
                # it is pure number ?
                char = str(int(char))
                # list of device only accept number with len > 15
                if len(char) >= 14:
                    list_device.append(char)
                else:
                    # add it to list of phone
                    list_phone.append(char)
            # if not pure number
            except ValueError:
                try: 
                    # could it find @ ?
                    char.index('@') 
                    # if so, then add it to list of email
                    list_email.append(char)
                except ValueError:
                    # else skip it 
                    pass
    return list_phone,list_email,list_device,list_index

def get_sub(S,i):
    ls = {}
    cell=set()
    for v1,v2,v3 in S[i].edges.data("weight"):
        try:
            ls[v3].append(v1)
            ls[v3].append(v2)
            ls[v3] = list(set(ls[v3]))
        except KeyError:
            ls[v3]=[v1,v2]
            
    for k,v in ls.items():
        ls[k] =[''.join(x) for x in classify_char(v)]
        
    sub = pd.DataFrame.from_dict(ls,orient='index').reset_index()
    sub.columns = ['index','phone','email','device_id','other']
    return sub

def whether_pair(sub):    
    by_phone = sub.sort_values(by = 'phone')
    by_email = sub.sort_values(by = 'email')
    by_device = sub.sort_values(by = 'device_id')
    
    scores = [by_score(by_phone),by_score(by_email),by_score(by_device)]
    df_res = []
    for x in scores:
        if x != None:
            df_res += x
    try:
        d = pd.concat(df_res,axis=0).drop_duplicates()
        return d
    except ValueError:
        pass


In [134]:
def layout_S(S,data):
# classify these values according Integrity
    # create 4 empty lists
    half, full, multi, weak_link = {},{},{},{}
    i,h,f,m,w = 0,1,1,1,1
    # for all in S except S[0], which includes all values linked with invalid value)
    while i <len(S):
        
        # classify values to a complete chain: cell_1: phone ,cell2: email, cell_3: device_id
        cell = classify_char(list(S[i].nodes))
        phone_str = ','.join(cell[0])
        email_str = ','.join(cell[1])
        device_str = ','.join(cell[2])
    
        cell_str = [phone_str,email_str,device_str]
        
        # if there is only 1 value in all 3 cells of a complete chain
        if len(cell[0]) + len(cell[1])+ len(cell[2]) <= 1:
            pass
        
        # if every cell has only 1 value
        elif (len(cell[0]) == 1) & (len(cell[1]) == 1) & (len(cell[2]) == 1):
            full['F'+str(f)] = cell_str
            f += 1
            
        # if there is cell more than 1 value
        elif (len(cell[0]) > 1) | (len(cell[1]) > 1) | (len(cell[2]) > 1):

            sub = get_sub(S,i)
            mlt = whether_pair(sub)
            try:
                len(mlt) 
                multi['M'+str(m)] =  mlt
                m += 1
            except TypeError:
                weak_link['W'+str(w)] = sub
                w += 1
            
        # at least one cell is empty
        else:
            half['H'+str(h)] = cell_str
            h += 1
        i += 1

    return half,full,multi,weak_link

# get new data

In [135]:
%%time
half, full, multi, weak_link = layout_S(S,dt)

Wall time: 7.1 s


In [136]:
%%time
df_weak_link = pd.concat(weak_link).reset_index().drop(['level_1','other'],axis=1)
df_weak_link.columns = ['new_index','index','phone','email','device_id']
df_weak_link

Wall time: 117 ms


,new_index,index,phone,email,device_id
0,W1,5465,9155450446,dharamrajes@gmail.com,355528113586656
1,W1,12006,9155450446,amankumardiwakar914@gmail.com,866410040208310
2,W1,3820,9955522559,dharmendrakumarkuju@gmail.com,351866093314374
3,W1,3822,9117787158,amankumardiwakar914@gmail.com,351866093314374
4,W2,1226,7678013640,amirkhanmohd5522@gmail.com,357567622607411
...,...,...,...,...,...
809,W397,49881,9711862286,,355528112335550
810,W398,50004,9450334130,,359574081200173
811,W398,50014,6394268707,,359574081200173
812,W399,50135,9845653722,,861192043198475


In [137]:
%%time
df_multi = pd.concat(multi).reset_index().drop(['level_1','other'],axis=1)
df_multi.columns = ['new_index','index','phone','email','device_id']
df_multi

Wall time: 300 ms


,new_index,index,phone,email,device_id
0,M1,14198,7675865822,dulisrinivsrao@yahoo.com,355528114419931
1,M1,1004,8019459961,dulisrinivsrao@yahoo.com,355528114419931
2,M2,6457,7518128287,sakshirules8@gmail.com,354406160787736
3,M2,3657,9161001208,sakhshirules8@gmail.com,354406160787736
4,M3,66,8903626752,srivallistudio@gmail.com,351826090443204
...,...,...,...,...,...
1693,M841,49879,9044127177,babaluguptanidi@gmail.com,
1694,M842,50506,8008228107,amit.agrawal.hyd@outlook.com,
1695,M842,50557,918008228107,amit.agrawal.hyd@outlook.com,
1696,M843,50513,9487514346,harivignesheie@gmail.com,


In [138]:
%%time
df_full = pd.DataFrame.from_dict(full,orient='index').reset_index()
df_full.columns = ['new_index','phone','email','device_id']
df_full

Wall time: 14.3 ms


,new_index,phone,email,device_id
0,F1,9538399595,jamesrlobo@gmail.com,357227100843414
1,F2,8329458713,paragmor24@gmail.com,355528113966437
2,F3,7979839174,prakashtravel19@yahoo.com,354406161386314
3,F4,9002224225,dhirajmkjee@gmail.com,354770922318555
4,F5,6203248152,rahul.mallik007@gmail.com,356206203076339
...,...,...,...,...
13664,F13665,7004164670,anuragnda2016@gmail.com,358557940804153
13665,F13666,9446502480,rohanslog@gmail.com,354770922275052
13666,F13667,8220884640,abisheksugu008@gmail.com,867062030730156
13667,F13668,9755739851,jpsahu1995@gmail.com,355528111511052


In [139]:
%%time
df_half = pd.DataFrame.from_dict(half,orient='index').reset_index()
df_half.columns = ['new_index','phone','email','device_id']
df_half

Wall time: 29.9 ms


,new_index,phone,email,device_id
0,H1,7404153238,gvindaku460@gmail.com,
1,H2,9434229832,,354143090533654
2,H3,9454419663,,869878034392331
3,H4,6200549947,,355637080856958
4,H5,8639978391,bvinay538@gmail.com,
...,...,...,...,...
34066,H34067,9189824223,rajeshshriwas448@gmail.com,
34067,H34068,9640264007,,351892081422771
34068,H34069,9582710157,9582710157@nomail.com,
34069,H34070,9881640670,,868416029393997


In [140]:
#    有多值  (包括缺失 1个         |    完整的    |   只有 2/3
#      强连接 ,         弱连接    |             |
len(df_multi),len(df_weak_link),len(df_full),len(df_half)

(1698, 814, 13669, 34071)

In [141]:
len(df_multi)+len(df_weak_link)+len(df_full)+len(df_half)

50252

In [143]:
13669/50252

0.27200907426570087